In [1]:
import sys
from pathlib import Path

def find_project_root(start_path: Path = Path.cwd(), marker: str = 'pyproject.toml') -> Path:
    current_path = start_path.resolve()
    for parent in [current_path] + list(current_path.parents):
        if (parent / marker).exists():
            return parent
        
def add_project_root_to_sys_path(marker: str = 'pyproject.toml'):
    project_root = find_project_root(marker=marker)
    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))

add_project_root_to_sys_path()


# Imports

In [2]:
import torch
import torch.nn as nn
import wandb
from tqdm import tqdm
from torch.utils.data import DataLoader
from src.asym_ensembles.data_loaders import load_dataset
from src.asym_ensembles.modeling.training import (
    set_global_seed,
    train_one_model,
    evaluate_model,
    evaluate_ensemble,
    average_pairwise_distance
)
from src.asym_ensembles.modeling.models import MLP, WMLP
import numpy as np
import copy

# Config

In [3]:
cfg={
    "batch_size": 64,
    "max_epochs": 200,
    "patience": 16,
    "learning_rate": 1e-3,
    "weight_decay": 3e-2,
    "hidden_dims": [64, 128, 256],
    "ensemble_sizes": [2, 4, 8, 16, 32, 64],
    "total_models": 64,             # max(ensemble_sizes)
    "repeats": 10,                  # different seeds
    "mask_type": "random_subsets",
    "base_seed": 1234,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

In [4]:
wandb.init(project="DeepEnsembleProject", config=cfg, name="Extended_Experiments", settings=wandb.Settings(start_method="fork"))
config = wandb.config

wandb: Currently logged in as: novitsk-oleg (oanovi). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


# Loading Data

In [13]:
all_datasets = [
    ("california", "regression"),
    ("otto", "classification"),
    ("telcom", "classification"),
    ("mnist", "classification"),
]
for dataset_name, task_type in all_datasets:
    train_ds, val_ds, test_ds = load_dataset(dataset_name=dataset_name)
    train_loader = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=config.batch_size, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=config.batch_size, shuffle=False)
    for hidden_dim in config.hidden_dims:
        print(f"\nDataset: {dataset_name}, Hidden_dim: {hidden_dim}")
        if hidden_dim in [64, 128]:
            second_nfix = 3
        else:
            second_nfix = 4
        mask_params = {
            0: {'mask_constant': 1, 'mask_type': config.mask_type, 'do_normal_mask': True, 'num_fixed': 2},
            1: {'mask_constant': 1, 'mask_type': config.mask_type, 'do_normal_mask': True, 'num_fixed': second_nfix},
            2: {'mask_constant': 1, 'mask_type': config.mask_type, 'do_normal_mask': True, 'num_fixed': second_nfix},
            3: {'mask_constant': 1, 'mask_type': config.mask_type, 'do_normal_mask': True, 'num_fixed': second_nfix},
        }
        in_dim = train_ds.tensors[0].shape[1]
        if task_type == "regression":
            out_dim = 1
            criterion = nn.MSELoss()
        else:
            out_dim = len(torch.unique(train_ds.tensors[1]))
            criterion = nn.CrossEntropyLoss()

        for rep_i in range(config.repeats):
            print(f"\nRepetition {rep_i + 1}/{config.repeats}")
            current_seed = config.base_seed + rep_i * 10000

            mlp_metrics = []
            wmlp_metrics = []
            wmlp_masked_ratios = []
            mlp_models = []
            wmlp_models = []

            for i in tqdm(range(config.total_models), desc="Training MLP"):
                seed_value = current_seed + i
                set_global_seed(seed_value)

                mlp = MLP(in_dim, hidden_dim, out_dim, num_layers=4, norm=None)
                optimizer = torch.optim.AdamW(mlp.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)

                mlp, train_time, train_losses, val_losses = train_one_model(
                    mlp, train_loader, val_loader, criterion, optimizer,
                    device=config.device, max_epochs=config.max_epochs, patience=config.patience
                )
                mlp.to("cpu")
                mlp_models.append(copy.deepcopy(mlp))

                test_metric = evaluate_model(mlp, test_loader, criterion, config.device, task_type=task_type)
                mlp_metrics.append(test_metric)

            for i in tqdm(range(config.total_models), desc="Training WMLP"):
                seed_value_wmlp = current_seed + 2000 + i
                set_global_seed(seed_value_wmlp)

                wmlp = WMLP(in_dim, hidden_dim, out_dim, num_layers=4, mask_params=mask_params, norm=None)
                optimizer_wmlp = torch.optim.AdamW(wmlp.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)

                wmlp, train_time_wmlp, train_losses_w, val_losses_w = train_one_model(
                    wmlp, train_loader, val_loader, criterion, optimizer_wmlp,
                    device=config.device, max_epochs=config.max_epochs, patience=config.patience
                )
                wmlp.to("cpu")
                wmlp_models.append(copy.deepcopy(wmlp))
                test_metric_wmlp = evaluate_model(wmlp, test_loader, criterion, config.device, task_type=task_type)
                wmlp_metrics.append(test_metric_wmlp)

                ratio, masked = wmlp.report_masked_ratio()
                wmlp_masked_ratios.append(ratio)

            avg_dist_mlp = average_pairwise_distance(mlp_models)
            avg_dist_wmlp = average_pairwise_distance(wmlp_models)

            avg_wmlp_masked_ratio = float(np.mean(wmlp_masked_ratios)) if wmlp_masked_ratios else 0.0

            ensemble_results_mlp = {}
            ensemble_results_wmlp = {}
            for ens_size in config.ensemble_sizes:
                mlp_sub = mlp_models[:ens_size]
                wmlp_sub = wmlp_models[:ens_size]
                ens_metric_mlp = evaluate_ensemble(mlp_sub, test_loader, config.device, task_type=task_type)
                ens_metric_wmlp = evaluate_ensemble(wmlp_sub, test_loader, config.device, task_type=task_type)
                ensemble_results_mlp[ens_size] = ens_metric_mlp
                ensemble_results_wmlp[ens_size] = ens_metric_wmlp

            log_dict = {
                "dataset_name": dataset_name,
                "task_type": task_type,
                "hidden_dim": hidden_dim,
                "repeat_index": rep_i + 1,
                "avg_dist_mlp": avg_dist_mlp,
                "avg_dist_wmlp": avg_dist_wmlp,
                "avg_wmlp_masked_ratio": avg_wmlp_masked_ratio,
                "mean_mlp_metric": float(np.mean(mlp_metrics)),
                "mean_wmlp_metric": float(np.mean(wmlp_metrics)),

            }

            for ens_size in config.ensemble_sizes:
                log_dict[f"mlp_ens_{ens_size}"] = ensemble_results_mlp[ens_size]
                log_dict[f"wmlp_ens_{ens_size}"] = ensemble_results_wmlp[ens_size]

            wandb.log(log_dict)

            print(f"Repetition {rep_i + 1}/{config.repeats} finished.")
            break
        break


Dataset: california, Hidden_dim: 64

Repetition 1/10


Training MLP:   2%|▏         | 1/64 [00:22<23:15, 22.15s/it]

Early stopping at epoch 102


Training MLP:   3%|▎         | 2/64 [00:47<24:46, 23.98s/it]

Early stopping at epoch 120


...

Early stopping at epoch 107


Training MLP:  81%|████████▏ | 52/64 [21:21<04:58, 24.90s/it]

Early stopping at epoch 108


Training MLP:  83%|████████▎ | 53/64 [21:48<04:41, 25.60s/it]

Early stopping at epoch 123


In [ ]:
wandb.finish()